In [1]:
import veloxchem as vlx
from openmmdynamics3 import OpenMMDynamics

In [2]:
first_xyz = '1.xyz' # Name of first xyz-file in quotes ex '1.xyz'
second_xyz = '2.xyz' # Name of second xyz-file in quotes ex '2.xyz'
identical_molecules = True # Set to True if the two molecules are identical, False if they are different
first_charge = 1 
second_charge = 1
first_multiplicity = 2
second_multiplicity = 2
medium = 'water' # gas or water
size_of_md_box = 4 # Cubic MD simulation box in nm
simulation_time = 1 # simulation time in nanoseconds

In [3]:
#Display the first molecule:
molecule1 = vlx.Molecule.read_xyz_file(first_xyz)
print('Structure of the molecule entered: ')
molecule1.show(atom_indices=True)

Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
#Dispaly the second molecule:
molecule2 = vlx.Molecule.read_xyz_file(second_xyz)
print('Structure of the molecule entered: ')
molecule2.show(atom_indices=True)

Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
#Generate force field and pdb for first structure
ff_gen = vlx.ForceFieldGenerator()
molecule1.set_multiplicity(first_multiplicity)
molecule1.set_charge(first_charge)
basis = vlx.MolecularBasis.read(molecule1, 'sto-3g')
scf_drv = vlx.ScfUnrestrictedDriver()
scf_drv.ostream.mute()
scf_drv.conv_thresh = 1e-3
scf_drv.max_iter = 500
scf_drv.xcfun = 'b3lyp'
results = scf_drv.compute(molecule1, basis)
ff_gen.ostream.mute()
ff_gen.create_topology(molecule1, basis , results)
ff_gen.write_openmm_files('mol1','MO1')

* Info * Reading basis set from file: /opt/miniconda3/envs/vlxenv/lib/python3.11/site-packages/veloxchem/basis/STO-3G     
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: STO-3G                                                                              
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          
                

In [7]:
#Generate force field and pdb for second structure
if identical_molecules == True:
    ff_gen2 = vlx.ForceFieldGenerator()
    ff_gen2.ostream.mute()
    ff_gen2.create_topology(molecule2, no_resp=True)
    ff_gen2.write_openmm_files('mol2','MO2')
else:
    ff_gen2 = vlx.ForceFieldGenerator()
    molecule2.set_multiplicity(second_multiplicity)
    molecule2.set_charge(second_charge)
    basis2 = vlx.MolecularBasis.read(molecule2, 'sto-3g')
    scf_drv2 = vlx.ScfUnrestrictedDriver()
    scf_drv2.ostream.mute()
    scf_drv2.conv_thresh = 1e-3
    scf_drv2.max_iter = 500
    scf_drv2.xcfun = 'b3lyp'
    results2 = scf_drv2.compute(molecule2, basis2)
    ff_gen2.ostream.mute()
    ff_gen2.create_topology(molecule2, basis2 , results2)
    ff_gen2.write_openmm_files('mol2','MO2')

In [8]:
#Run molecular dynamics
opm_dyn = OpenMMDynamics()
opm_dyn.box_size = size_of_md_box
pdb_files = ['mol1.pdb', 'mol2.pdb']
xml_files = ['mol1.xml']
opm_dyn.build_custom_system(pdb_files, xml_files, phase=medium)
steps=simulation_time*500000
snap=200

# Vi kan nu köra en kort simulering för att se att allt fungerar!!!
opm_dyn.run_md(ensemble='NVT', 
               temperature=300,
               timestep=2.0,
               nsteps=steps,
               snapshots=snap)

Running simulation...
Ensemble: NVT
Temperature: 300 K
Friction: 1.0 1/ps
Timestep: 2.0 fs
Total simulation time in ns: 1.0
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Density (g/mL)"
2500,-93331.22928221116,304.23801523151974,0.96595523968614
5000,-93020.06106932054,294.9149730407111,0.96595523968614
7500,-92482.11319334398,293.2394886669712,0.96595523968614
10000,-92341.44382078538,301.3559670362773,0.96595523968614
12500,-92358.2921484221,300.46969194861515,0.96595523968614
15000,-92575.66507322679,304.98217921626855,0.96595523968614
17500,-92186.00528318773,298.76505794707936,0.96595523968614
20000,-92573.3214452971,298.08102897643715,0.96595523968614
22500,-92360.09952146898,292.00931698770444,0.96595523968614
25000,-92478.14541990648,302.6258968784747,0.96595523968614
27500,-92654.54678709398,295.39111688768253,0.96595523968614
30000,-92763.46585447679,300.23487158830574,0.96595523968614
32500,-92585.49856932054,297.8247423852045,0.96595523968614
35000,-93066.00210935

In [9]:
#Visualize dynamics
import nglview
import mdtraj as md

traj = md.load("trajectory.pdb", top="trajectory.pdb")
traj.image_molecules(anchor_molecules=[set(traj.topology.residue(0).atoms)])
view = nglview.show_mdtraj(traj)
view.clear_representations()
view.add_representation("ball+stick", selection="water", opacity=0.2)
view.add_spacefill("not water")
view.center()
view

NGLWidget(max_frame=199)

In [10]:
print(results['F'])

(array([[-1.89851584e+01, -4.88774175e+00,  7.47541475e-08, ...,
        -2.04698639e-02, -1.34876983e-05,  1.45660757e-01],
       [-4.88774175e+00, -2.19295251e+00, -1.63705678e-01, ...,
        -2.63563329e-02,  1.83375125e-02,  1.86330004e-01],
       [ 7.47541475e-08, -1.63705678e-01, -7.78165362e+02, ...,
        -6.08911182e-07,  4.17555572e-05,  7.55265950e-07],
       ...,
       [-2.04698639e-02, -2.63563329e-02, -6.08911182e-07, ...,
        -8.58819915e-01, -2.95087240e-03,  7.80407287e-03],
       [-1.34876983e-05,  1.83375125e-02,  4.17555572e-05, ...,
        -2.95087240e-03, -1.00599262e+01, -2.53410618e+00],
       [ 1.45660757e-01,  1.86330004e-01,  7.55265950e-07, ...,
         7.80407287e-03, -2.53410618e+00, -8.99375849e-01]]), array([[-1.89752616e+01, -4.87764544e+00,  5.91135121e-08, ...,
        -2.04136248e-02, -1.34954887e-05,  1.45387375e-01],
       [-4.87764544e+00, -2.14092382e+00, -1.63705634e-01, ...,
        -2.64068901e-02,  1.83145581e-02,  1.84719705